In [ ]:
## Import modules
import pandas as pd
import numpy as np

import datetime
from   datetime import datetime, date
pd.set_option('display.notebook_repr_html', False)
pd.set_option('display.max_columns', 7)
pd.set_option('display.max_rows', 10)
pd.set_option('display.width', 60)

import matplotlib.pyplot as plt
get_ipython().run_line_magic('matplotlib', 'inline')

import os
from os import listdir
from os.path import isfile, join
import glob
import re

import warnings
warnings.filterwarnings("ignore")

In [ ]:
## Define the path of files

filepath_mnth = []
for file in glob.glob("//10.64.187.59/part1/[0-9]*"):
# for file in glob.glob("Z:/"):
    filepath_mnth.append(file)

filepath_mnth_re = filepath_mnth[4:]+filepath_mnth[:3]
filepath_mnth_re

In [ ]:
filepath = []
# for file in glob.glob("S:/2차펌프(1공장)/P301A*.csv"):
# for file in glob.glob("//10.64.187.59/part1/1월/2차펌프(1공장)/P3*.csv"):
for file in glob.glob(filepath_mnth_re[0]+"/2차펌프(1공장)/*"):
# for file in glob.glob("Z:/"):
    filepath.append(file)

filepath

In [ ]:
## Collect the path of files by months respectively 

filedic = dict()
filepath = []

for i in range(len(filepath_mnth_re)):
    for file in glob.glob(filepath_mnth_re[i]+"/2차펌프(1공장)/*.csv"):
        filepath.append(file)
    filedic[i] = filepath
    filepath = []
    print(i)

## Define the name of pumps of 2nd factory

pump_name_2nd_factory = []
for r in list('ABCDEFGHIJKLMNOP'):
    pump_name_2nd_factory.append('P301'+r)

In [ ]:
## Make dictionary form

pump_dic = dict()
pump_path = []
for j in range(len(pump_name_2nd_factory)):
    for i in range(len(filedic)):
        for r in filedic[i]:
            if bool(re.search(pump_name_2nd_factory[j], r)):    
                pump_path.append(r)
            pass
    pump_dic[j] = pump_path
    pump_path = []

In [ ]:
## Data Load
use_cols = ['Time', 'Value']
datasets = dict()
grouped_datasets = dict()
no = 0 # the length of pump_dic is 16
filelist = pump_dic[no]
for i in range(len(filelist)):
    data = pd.read_csv(filelist[i], usecols = use_cols)
    if len(data) != 0:
#         data.columns = ['Time', filelist[i][13:-4]]
        data.columns = ['Time', filelist[i][filelist[i].find('\\')+1:-4]]
        data['Time'] = pd.to_datetime(data['Time'], format = "%Y/%m/%d %H:%M:%S.%f")
        data['Time_conv'] = pd.to_datetime(data['Time'].map(lambda x: x.strftime('%Y/%m/%d %H:%M')))
        datasets[i] = data
        grouped_data = datasets[i].groupby('Time_conv', as_index=False).mean()
        grouped_datasets[i] = grouped_data    
        print("********** ", i+1, "th dataset / total", len(filelist), "dataset is done **********")
    else: print("********** ", i+1, "th dataset is empty **********")

In [ ]:
full_data_index = dict()
data_index = []
pump_name = pump_name_2nd_factory[0]
for j in range(len(features_dic[pump_name])):
    for i in range(len(grouped_datasets)):
        try:
            data_index.append(bool(re.search('_'+features_dic[pump_name][j], str(grouped_datasets[i].columns))))
        except KeyError:
            data_index.append(False)
    full_data_index[j] = data_index
    data_index = []

pump_features_dic = dict()
foo = []
for j in range(len(full_data_index)):
    for index in list(np.where(full_data_index[j])[0]):
        foo.append(grouped_datasets[index])
    pump_features_dic[j] = pd.concat(foo)
    foo = []
    
for merge_pump_name in pump_name_2nd_factory:
    full_data_index = dict()
    data_index = []
    pump_name = merge_pump_name
    for j in range(len(features_dic[pump_name])):
        for i in range(len(grouped_datasets)):
            try:
                data_index.append(bool(re.search('_'+features_dic[pump_name][j], str(grouped_datasets[i].columns))))
            except KeyError:
                data_index.append(False)
        full_data_index[j] = data_index
        data_index = []
    
    pump_features_dic = dict()
    foo = []
    for j in range(len(full_data_index)):
        for index in list(np.where(full_data_index[j])[0]):
            foo.append(grouped_datasets[index])
        pump_features_dic[j] = pd.concat(foo)
        foo = []

In [ ]:
## Make data market for merge(period = 1min)
tmp_date = pd.date_range(start='12/31/2013', end='12/31/2014')
tmp_df_date = pd.DataFrame(tmp_date, columns=['date'])
tmp_df_date['date'] = tmp_df_date['date'].astype(str)
tmp_df_date['date'] = tmp_df_date['date'].str[0:10]

tmp_time = pd.date_range(start='00:00:00', end='23:59:00', freq='1min')
tmp_df_time = pd.DataFrame(tmp_time, columns=['time'])
tmp_df_time['time'] = tmp_df_time['time'].astype(str)
tmp_df_time['time'] = tmp_df_time['time'].str[10:19]

tmp_init_DT = pd.DataFrame(columns=['date', 'time'])
tmp_init_DT2 = pd.DataFrame(columns=['date', 'time'])

for i in range(0,366):
    if i == 0:
        tmp_init_DT['time'] = tmp_df_time['time']
        tmp_init_DT['date'] = tmp_df_date['date'][0]
    else:
        tmp_init_DT2['time'] = tmp_df_time['time']
        tmp_init_DT2['date'] = tmp_df_date['date'][i]
        tmp_init_DT = tmp_init_DT.append(tmp_init_DT2)

del tmp_df_time, tmp_df_date, tmp_init_DT2

tmp_init_DT['Time_conv'] = tmp_init_DT['date']+tmp_init_DT['time']
tmp_init_DT['Time_conv'] = pd.to_datetime(tmp_init_DT['Time_conv'], format = "%Y/%m/%d %H:%M:%S.%f")
tmp_init_DT = tmp_init_DT.drop(columns= ['date', 'time'])
tmp_init_DT = tmp_init_DT[900:-540]
data_market = tmp_init_DT
del tmp_init_DT

In [ ]:
## Merging(Left Join)

merge_pump_name = pump_name_2nd_factory[0]
merging_result = data_market
for i in range(len(pump_features_dic)):
    merging_result = pd.merge(merging_result, pump_features_dic[i], how = 'left', on = 'Time_conv')
merging_result.to_csv('uniset_'+merge_pump_name+'.csv')
print(merge_pump_name+'equipment file merge complete')


In [ ]:
## Final Merging Code
## Only use this code for data load & merging
for m in range(len(pump_name_2nd_factory)):
    
    use_cols = ['Time', 'Value']
    datasets = dict()
    grouped_datasets = dict()
    no = m # the length of pump_dic is 16
    filelist = pump_dic[no]
    for i in range(len(filelist)):
        data = pd.read_csv(filelist[i], usecols = use_cols)
        if len(data) != 0:
    #         data.columns = ['Time', filelist[i][13:-4]]
            data.columns = ['Time', filelist[i][filelist[i].find('\\')+1:-4]]
            data['Time'] = pd.to_datetime(data['Time'], format = "%Y/%m/%d %H:%M:%S.%f")
            data['Time_conv'] = pd.to_datetime(data['Time'].map(lambda x: x.strftime('%Y/%m/%d %H:%M')))
            datasets[i] = data
            grouped_data = datasets[i].groupby('Time_conv', as_index=False).mean()
            grouped_datasets[i] = grouped_data    
            print("********** ", i+1, "th dataset / total", len(filelist), "dataset is done **********")
        else: print("********** ", i+1, "th dataset is empty **********")
    
    
    full_data_index = dict()
    data_index = []
    pump_name = pump_name_2nd_factory[m]
    for j in range(len(features_dic[pump_name])):
        for i in range(len(grouped_datasets)):
            try:
                data_index.append(bool(re.search('_'+features_dic[pump_name][j], str(grouped_datasets[i].columns))))
            except KeyError:
                data_index.append(False)
        full_data_index[j] = data_index
        data_index = []
    
    pump_features_dic = dict()
    foo = []
    for j in range(len(full_data_index)):
        for index in list(np.where(full_data_index[j])[0]):
            foo.append(grouped_datasets[index])
        pump_features_dic[j] = pd.concat(foo)
        foo = []
        
    merging_result = data_market
    for k in range(len(pump_features_dic)):
        merging_result = pd.merge(merging_result, pump_features_dic[k], how = 'left', on = 'Time_conv')
    merging_result.to_csv('uniset_'+pump_name_2nd_factory[m]+'.csv')
    print(pump_name_2nd_factory[m]+'장비의 merge file 제작 완료')